<a href="https://colab.research.google.com/github/AchiengMary/CubeSat_Nairobi/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential  # Importing Sequential to build the model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense  # Importing necessary layers for the CNN

from keras.utils import to_categorical

# One-hot encode the labels (assuming you have 5 classes)
train_labels = to_categorical(train_labels, num_classes=5)

# Define the CNN model architecture
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(512, 512, 3)),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    Conv2D(32, (3, 3), activation='relu'),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Convolutional layer + ReLU activation
    MaxPooling2D((2, 2)),  # Max pooling layer
    GlobalAveragePooling2D(),  # Global average pooling layer
    Dense(5, activation='softmax')  # Output layer with 5 neurons (one for each class) + Softmax activation
])

# Compile the model with appropriate loss function, optimizer, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Model defined and compiled successfully.")

# Train the model on the training data
history = model.fit(
    train_images, train_labels,
    epochs=10,  # Number of epochs
    batch_size=64,  # Batch size
)

print("Model training complete.")

NameError: name 'train_labels' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess dataset (CIFAR-10 as an example)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize images to [0,1] range
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
num_classes = 10  # Adjust if using a different dataset
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Data Augmentation for better generalization
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

# Load pre-trained MobileNetV3 without top layers
base_model = MobileNetV3Small(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers to retain pre-trained features
base_model.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)  # Global pooling to reduce dimensions
x = Dense(128, activation="relu")(x)  # Fully connected layer
x = Dropout(0.3)(x)  # Dropout to prevent overfitting
output_layer = Dense(num_classes, activation="softmax")(x)  # Output layer

# Create final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train the model for 15 epochs
history = model.fit(train_generator,
                    validation_data=(x_test, y_test),
                    epochs=15,
                    batch_size=32,
                    verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the trained model
model.save("mobilenetv3_cifar10.h5")
print("Model saved successfully!")

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Load the dataset from .npy files
x_train = np.load("data/train_images.npy")
y_train = np.load("data/train_labels.npy")
x_test = np.load("data/test_images.npy")
y_test = np.load("data/test_labels.npy")

#Normalize images (convert pixel values to range [0,1])
x_train, x_test = x_train / 255.0, x_test / 255.0

#Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))  # Automatically detect number of classes
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Data Augmentation for robustness
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

#Load pre-trained MobileNetV3 as a feature extractor
base_model = MobileNetV3Small(weights="imagenet", include_top=False, input_shape=x_train.shape[1:])

#Freeze base model layers to retain pre-trained features
base_model.trainable = False

#Build a custom classification head
x = GlobalAveragePooling2D()(base_model.output)  # Reduce dimensions
x = Dense(128, activation="relu")(x)  # Fully connected layer
x = Dropout(0.3)(x)  # Prevent overfitting
output_layer = Dense(num_classes, activation="softmax")(x)  # Output layer

#Create final model
model = Model(inputs=base_model.input, outputs=output_layer)

#Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

#Train the model for 15 epochs
history = model.fit(train_generator,
                    validation_data=(x_test, y_test),
                    epochs=3,
                    batch_size=32,
                    verbose=1)

#Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")

#Save the trained model
model.save("mobilenetv3_custom.h5")
print("Model saved successfully!")


FileNotFoundError: [Errno 2] No such file or directory: 'data/train_images.npy'